
### Created: July, 2023
### Authors: T. A. Biala, Y.O. Afolabi and I. O. Sarumi

This function implements a rational approximation to the Prabhakar Function (Three Parameter  Mittag Leffler (ML) function) with 1, 2, or 3 parameters. 
The function is named RA which stands for rational approximation. See [1] for a full discussion of the approximation.

###### Inputs
- $z$: the vector (or matrix) whose Prabhakar function we wish to compute
- $m$: the number of terms (to use) of the local series expansion of the Prabhakar function
- $n$: the number of terms of the asymptotic expansion of the Prabhakar function
- $\alpha$: the alpha parameter in the Prabhakar function
- $\beta$: the beta parameter in the Prabhakar function
-  $\gamma$: the gamma parameter in the Prabhakar function. $r=\gamma$ throughout the notebook

###### Outputs:
- Returns a dictionary with 3 items: 
    - parms: the parameters used in the computation of the approximation, see [1].
    - coeffs: the coefficients of the $(\nu, \nu)$-rational approximation, where $\nu = \dfrac{m+n+r-2}{2}$
    - z_RA: the evaluated rational approximation for the elements of $z$


RA($z, m, n, \alpha$) computes the rational approximation with one parameter ($\alpha$) for $z$; $\alpha$ must be real and positive. The one parameter ML function is defined as 
$$ E_{\alpha}(z) = \sum_{k=0}^{\infty} \dfrac{z^k}{\Gamma(\alpha k+1)},$$
where $\Gamma$ is the Euler gamma function.

RA($z, m, n, \alpha, \beta$) computes the rational approximation with two parameters ($\alpha, \beta$) for $z$; $\alpha$ must be real and positive, $\beta$ must be a real scalar. The two parameter ML function is defined as 
$$ E_{\alpha, \beta}(z) = \sum_{k=0}^{\infty} \dfrac{z^k}{\Gamma(\alpha k+\beta)}.$$


RA($z, m, n, \alpha, \beta, \gamma$) computes the rational approximation with three parameters ($\alpha, \beta, \gamma$) for $z$; $\alpha$ must be real and positive, $\beta$ must be a real scalar and $\gamma$ is a positive integer. The three parameter ML function is defined as 
$$ E_{\alpha, \beta}^\gamma(z) = \sum_{k=0}^{\infty} \dfrac{\Gamma(\gamma + k)z^k}{\Gamma(k+1)\Gamma(\alpha k+\beta)}.$$





In [1]:
# Importing Libraries
import numpy as np
import scipy.special as sc
import scipy.linalg as sl
import numpy.linalg as nl
import math
import warnings

###### Helper Functions

- Function **c_coeffs** computes the C-coefficient,  see [1] for details,  $$C_{\alpha, \beta}^r = \dfrac{\Gamma(\beta - \alpha r)}{\Gamma(r)}$$.

In [2]:
 def c_coeffs(alpha, beta, r):
    return sc.gamma(beta - alpha * r)/sc.gamma(r)

- Function **b_coeffs** computes the $\mathbf{b}$-coefficients, see [1] for details, $$b_{r+j} = \dfrac{(-1)^j\Gamma(r+j)}{\Gamma(j+1)\Gamma(\alpha j + \beta)}C_{\alpha, \beta}^r, ~ j=0, \cdots, m-2,$$
where $b_j = 0, ~j=0, \cdots, r-1$.

In [3]:
def b_coeffs(alpha, beta, r, m):
    c_coeff = c_coeffs(alpha, beta, r)
    j = np.arange(m)
    return  (-1)**(j+1)*sc.gamma(j+r)/(sc.gamma(j+1)*sc.gamma(alpha*j + beta))*c_coeff

- Function **a_coeffs** computes the $\mathbf{a}$-coefficients, see [1] for details, $$a_{r+j} = \dfrac{(-1)^j\Gamma(r+j)}{\Gamma(j+1)\Gamma(\beta - \alpha j)}C_{\alpha, \beta}^r, ~ j=0, \cdots, n-1,$$
where $a_j = 0, ~j=0, \cdots, r-1$.

In [4]:
def a_coeffs(alpha, beta, r, n):
    c_coeff = c_coeffs(alpha, beta, r)
    j = np.arange(n)
    return  (-1)**(j+1)*sc.gamma(j+r)/(sc.gamma(j+1)*sc.gamma(beta - alpha*(j+r)))*c_coeff

- Function **top_set** uses two other functions: **top_right** and **top_left**
    - **top_left** is an $(m-1, \nu-r)$ rectangular diagonal matrix with entries $1$
    - **top_right** is a 
        - $(\nu, \nu)$ matrix if $\nu-r-n+1 = 0\implies \nu = m-1$,
        - $(2\nu-r-n+1, \nu)$ matrix if  $\nu-r-n+1 > 0$  
    - **top_left** and **top_right** are stacked horizontally to obtain **top_set**

In [5]:
def top_left(num, nu, r):
    return np.eye(num, nu-r)

def top_right(b, nu, n, r):
    top =  np.tril(sl.toeplitz(tuple(b[:nu])))
    nunr1 = nu-n-r+1
    if nunr1 > 0:
        ll = [b[i:i+nu][::-1] for i in range(1, nunr1+1)]
        top = np.vstack((top, ll))
    return top      
    
def top_set(num, nu, n, r, b):
    left = top_left(num, nu, r)
    right = top_right(b, nu, n, r)
    return np.hstack((left, right))

- Function **bottom_set** uses two other functions: **bottom_right** and **bottom_left**
    - **bottom_left** is an $(n-1, \nu-r)$-rectangular antidiagonal matrix with entries $1$ 
    - **bottom_right** is 
        - $(n-1, n-1)$-lower antitriangular matrix with entries $a_{r+j}, \tiny{j = 0, \cdots, n-2}$ if $\nu-n+1 = 0\implies \nu = n-1$,    
        - $(n-1, \nu)$ which consists of $(n-1, n-1)$-lower antitriangular matrix with entries $a_{r+j}, \tiny{j = 0, \cdots, n-2}$ horizontally stacked with $(n-1, \nu-n+1)$-zero matrix if  $\nu-n+1 > 0$ 
    - **bottom_left** and **bottom_right** are stacked horizontally to obtain **bottom_set**

In [6]:
def bottom_left(num, nu, n, r):
    return np.eye(num, nu-r)[:, ::-1]

def bottom_right(num, nu, n, r, a):
    nun1 = nu-n+1
    temp = np.tril(sl.toeplitz(tuple(a[:num])))[:, ::-1]
    if nun1 > 0:
        return np.hstack((np.zeros((num, nun1)), temp))
    return temp
    
def bottom_set(num, nu, n, r, a):
    left = bottom_left(num, nu, n, r)
    right = bottom_right(num, nu, n, r, a)
    return np.hstack((left, right))

- Function **create_matrix** consists of vertically stacked results from **top_set** and **bottom_set**

In [7]:
def create_matrix(nu, m, n, r, a_coeff, b_coeff):   
    top_num, bottom_num = m-1, n-1 
    top = top_set(top_num, nu, n, r, b_coeff)
    bottom = bottom_set(bottom_num, nu, n, r, a_coeff)
    return np.vstack((top, bottom))

- Function **create_vector** returns a vector $\in \mathbb{R}^{2\nu-r}$  which consists of 
    - 0-vector $\in \mathbb{R}^{\nu-r}$
    - a vector $\in \mathbb{R}^{1}$ with entry -$1$
    - 0-vector $\in \mathbb{R}^{r-1}$
    - a vector $\in \mathbb{R}^{\nu-r-n+1}$  with entries -$b_{r+j}, \tiny{j = 0, \cdots, \nu-r-n}$, if $\nu-r-n+1>0$
    - a vector $\in \mathbb{R}^{n-1}$  with entries -$a_{r+j}, \tiny{j = 1, \cdots, n-1}$

In [8]:
def create_vector(nu, n, r, a, b):
    a = list(-a) 
    b = list(-b)
    nunr1 = nu-n-r+1
    nur = nu-r
    r1 = r-1
    temp = [0]*nur + [-1.0] + [0]*r1 
    if nunr1 > 0:
        temp += b[:nunr1]
    temp += a[1:n]
    return np.array(temp)

- Function **sol_vars** returns the solution variables as a dictionary with the coefficients $\{p_{_k}, q_{_k}\}$ as keys and their corresponding solution values as values
- Function **RA_approx** returns a vector of  approximations to the elements of $z$ (flattened if matrix)
- Function **RA** returns  a dictionary of a triple consisting of the parameters used in the rational approximation, the coefficients of the rational approximation and a vector or matrix (depending on $z$) of  approximations to the elements of $z$.

In [9]:
def sol_vars(sol, nu, m, n, alpha, beta, r):
    nur = nu-r
    coeffs = {'alpha' : alpha,
              'beta' : beta,
              'nu' : nu,
              'm' : m,
              'n' : n,
              'gamma' : r,
              'r' : r}
    sol_dict = {}
    for i in range(r):
        sol_dict['p' + str(i)] = 0.0
    for i in range(nur):
        sol_dict['p' + str(r+i)] = sol[i]
    for i in range(nu):
        sol_dict['q' + str(i)] = sol[nur+i]
    sol_dict['p' + str(nu)] = 1.0
    sol_dict['q' + str(nu)] = 1.0
    return coeffs, sol_dict

def RA_approx(x, sol_dict, alpha, beta, r):
    nu = int(len(sol_dict)/2)
    num, den = 0, 0
    for i in range(nu-1):
        num += sol_dict['p' + str(i)]*x**i
        den += sol_dict['q' + str(i)]*x**(i)
    num += sol_dict['p' + str(nu-1)]*x**(nu-1)
    den += sol_dict['q' + str(nu-1)]*x**(nu-1)
    den *= sc.gamma(beta-alpha*r)*x**r
    return num/den

def RA(z, m, n, alpha, beta=1, gamma=1):

    r = gamma 
    nu =  math.floor((m + n + r - 2)/2)
    
    assert type(gamma) == int, f'GAMMA = {GAMMA} must be an integer'
    assert beta != alpha * r, f'BETA = ALPHA * GAMMA, i.e. {beta} = {alpha} * {r}'
    assert (m+n+r)%2 == 0, f'm+n+r is not even, i.e, {m}+{n}+{r} = {m+n+r} is odd'
    assert nu < m, f'nu={nu} must be less than m={m}'
    if isinstance(z, (list, tuple, np.ndarray)):
        z = np.array(z)
    else:
        raise Exception('z must be of type list, tuple,  or nd.array')
    if (z < 0).any():
        z = np.abs(z)
        warnings.warn('\nInput z contains negative values. The absolute values of z will be used instead.')
    
    # Get the shape of z and flatten for easy computation
    z_shape = z.shape
    z = z.flatten()
    
    # Check if there are zero elements in z and compute this value separately
    zero_indices = np.where(z==0)[0]
    zero_val = 1.0/sc.gamma(beta)
    if zero_indices.shape[0] != 0:
        z = np.delete(z, zero_indices)

    b_coeff = b_coeffs(alpha, beta, r, m)
    a_coeff = a_coeffs(alpha, beta, r, n)
    
    A = create_matrix(nu, m, n, r, a_coeff, b_coeff)
    b = create_vector(nu, n, r, a_coeff, b_coeff)
    
    sol = np.linalg.solve(A, b)
  
    parms, coeffs = sol_vars(sol, nu, m, n, alpha, beta, r)
    sol_vector = RA_approx(z, coeffs, alpha, beta, r)
    
    # insert the ML value of zero elements using the appropriate indices
    for i in zero_indices:
        sol_vector = np.insert(sol_vector, i, zero_val)
        
    # Finally, reshape solution to have same size as input
    z_RA = sol_vector.reshape(z_shape)
    
    return {'parms': parms, 'coeffs': coeffs, 'z_RA': z_RA}


#### References
1. Y. O. Afolabi, T. A. Biala, I. O. Sarumi and B. A. Wade, `Global-Type Rational Approximations of the Three-Parameter Mittag-Leffler Functions Based on Polynomial Bases Functions` Submitted